# 4. Registration

# Why all the fuss? 

We need to be able to be able to generalize our findings. Both within our sample and overall. To do this we need to treat each scan like silly putty that isn't very stretchy.

# Space the final frontier

We talk about a couple of general spaces
1. Native space
2. Standard Space

## Native Space
The subject's personal space. If you look at enough images you realize these can be wildly different.

## Standard Space

These are defined spaces.

1. Talairach
 - Don't use
 - Based on a single woman
 - Please don't
2. MNI
 - Based on 152 brains
 - Now a lot of different flavors (asym) 

# Steps for carrying out transformation
1. Choose the type of transformation
2. Estimate the parameters of the transformation
 - Requires a cost function to evaluate parameter choice
3. Resample the original image
 - Transformed coordinates will not line up exactly with original coordinates
 - Use some type of interpolation

## Choose the type of transformation
1. Rigid body (6 DOF)
 - within-subject motion
2. Non-linear (lots of DOF!)
 - high-quality image (resolution, contrast) & same modality of reference/template
 - better with a non-linear template (e.g. MNI152_T1_2mm)
3. Affine (12 DOF)
 - needed as a starting point for non-linear
 - align to affine template, or using lower quality images, or eddy current correction
4. Global scaling (7 DOF)
- within-subject but with global scaling (equal in x,y,z)
- corrects for scanner scaling drift in longitudinal studies
#### More DOF is NOT always better (e.g. within-subject)

![affine](https://github.com/grace-shearrer/fMRI_bootcamp2019/blob/wildwest/4dock/images/affine.png?raw=true)

![spacey](https://github.com/grace-shearrer/fMRI_bootcamp2019/blob/wildwest/4dock/images/otherspaces.png?raw=true)

# Step 1
![step1](https://github.com/grace-shearrer/fMRI_bootcamp2019/blob/wildwest/4dock/images/step1.png?raw=true)

# Step 2
![step2](https://github.com/grace-shearrer/fMRI_bootcamp2019/blob/wildwest/4dock/images/step2.png?raw=true)

# Step 3
![step3](https://github.com/grace-shearrer/fMRI_bootcamp2019/blob/wildwest/4dock/images/step3.png?raw=true)

# All together
![sofine](https://github.com/grace-shearrer/fMRI_bootcamp2019/blob/wildwest/4dock/images/sofine.png?raw=true)

# Nonlinear is where things get weird
You have unlimited degrees of freedom and you essentially deforming a field.   
I like to think I know a lot about MRI, but FNIRT is not one of them
![warp](https://github.com/grace-shearrer/fMRI_bootcamp2019/blob/wildwest/4dock/images/warp.png?raw=true)


## Estimate the parameters of the transformation
Or there is no free lunch  

If you start pulling the brain like silly putty eventually it can become a big mess... But how big of a mess?

We are measuring the 'goodness of fit'


## FLIRT FLAVORS
1. Least Squares 
 - Same modality
 - EXACT SAME (including brightness)
2. Normalised Correlation 
 - Same modality
 - can change brightness & contrast
 - What MCFLIRT uses
3. Correlation Ratio 
 - Any MR modalities
 - FSL standard
4. Mutual Information 
 - Any modalities
 - Including CT, PET, etc.
5. Normalised Mutual Info. 
 - Any modalities
 - including CT, PET, etc.
6. BBR 
 - Within-subject EPI to structural
 - see later

## Resample the original image (interpolation)
Try as we might we won't line up each voxel exactly. SO how can we estimate which is voxel goes where?

1. Nearest Neighbor
 - The value of the new voxel is the nearest voxel from the original image
 - Lose resolution (images look blocky)
 - Useful when transformed image values must match original image
 - Mask images –  Atlases
2. Linear interpolation (trilinear)
 - Take a weighted average of transformed voxels
 - Integrates over nearest 8 voxels in 3d •  Fast, but may blur the image  
3. Higher Order Interpolation •  Sinc interpolation
 - A windowed sinc function is typically used

# Concept Check:

You have two  different types of images from
the same subject: T1-weighted and T2-weighted images

How can you align the images so that they can be
used for multi-modal segmentation? How many DOF?

## Rigid body 6-DOF
The key here is it is the _same subject_ so we only need 6 DOF to align the images cause it is the _same brain_

# Concept Check:
What cost function should you use?

## Correlation or Normalized Mutual or Mutual Information
The key here is you are registering two different *modalities*

# Registration can go horribly wrong
Ya gotta look at your images

# In FMRIPREP
We haven't chatted a lot about fMRIPREP here cause well they do it the hard way. Which is great cause they took the hard part and automated it! 

To register the EPI through to native space they use BBR (boundry based registration). 


# Concept check: How many DOF do you think they use given they are using FLIRT?

## 6-DOF! Same subject :)

## To standard space
### Who can translate this?

This workflow concatenates the transforms calculated upstream (see Head-motion estimation, Susceptibility Distortion Correction (SDC) –if fieldmaps are available–, _EPI to T1w registration_, and an _anatomical-to-standard transform_ from T1w/T2w preprocessing) to map the EPI image to the standard spaces given by the --output-spaces argument (see Defining standard and nonstandard spaces where data will be resampled). It also maps the T1w-based mask to each of those standard spaces.

Transforms are concatenated and applied all at once, with one interpolation (Lanczos) step, so as little information is lost as possible.

The output space grid can be specified using modifiers to the --output-spaces argument.

https://fmriprep.readthedocs.io/en/stable/_static/sample_report.html

# What does this look like in FSL FEAT?

## Example func to High res
![x2res](https://github.com/grace-shearrer/fMRI_bootcamp2019/blob/wildwest/4dock/images/ex2hires.png?raw=true)

### Note: The example_func is the image used to align all other images in MCFLIRT!

## High res to standard (MNI-152)
![hi2stand](https://github.com/grace-shearrer/fMRI_bootcamp2019/blob/wildwest/4dock/images/hiRes2stand.png?raw=true)

## Example_func to standard (MNI-152)
![x2stand](https://github.com/grace-shearrer/fMRI_bootcamp2019/blob/wildwest/4dock/images/exfunc2standard.png?raw=true)

# NOTE: IF YOU ARE IMPORTING FMRIPREP DATA TO FEAT YOU NEED TO WORK AROUND YOUR REGISTRATION

FEAT can do its thing cause it assumes you have run all your analyses in FEAT. When we pick and choose FEAT starts to miss folders and will fail. One such folder is the reg folder in lower level FEAT directories.

In [6]:
import os
import glob
import shutil

def reg_check(basedir,IDmat):
    for sub in glob.glob(os.path.join(basedir,'sub-*','grace_edit','*.feat')):
        
        if os.path.exists(os.path.join(sub,'reg_standard')):
            shutil.rmtree(os.path.join(sub,'reg_standard'))
            print('%s has the reg standard'%sub)
        
        for files in glob.glob(os.path.join(sub,'reg','*.mat')):
            print(files)
            os.remove(files)
        
        shutil.copy2(IDmat,os.path.join(sub,'reg'))
        meanFUNC=os.path.join(sub,'mean_func.nii.gz')
        regDIR=os.path.join(sub,'reg','standard.nii.gz')
        shutil.copy2(meanFUNC,regDIR)
        
                    
        
            
def main(BASEDIR):
    basedir=BASEDIR
    IDmat='/usr/local/fsl/etc/flirtsch/ident.mat'
    reg_check(basedir,IDmat)

if __name__ == "__main__":
    BASEDIR='/Users/gracer/Desktop/data/derivatives/task'
    main(BASEDIR)

# check that reg_standard exists in the first level
# if it exists delete:
# delete all .mat files
# copy the mean_func.nii.gz to reg/standard.nii.gz
#check the voxel intensities in the stats/cope.nii.gz and reg_standard/stats/cope.nii.gz 
# need to be the same (level 1)
# data dimension and pixel size should be the same

https://fsl.fmrib.ox.ac.uk/fslcourse/lectures/practicals/registration/index.html  
https://fsl.fmrib.ox.ac.uk/fslcourse/lectures/reg.pdf  
Jeannette Mumford's UT Austin Lecture notes

# End Registration
Take break